In [77]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [78]:
using Revise
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [79]:
includet("../VARs/utils.jl")
includet("../VARs/VAR_OLS.jl")

In [80]:
df = DataFrame(XLSX.readtable("../../data/usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

292×11 TimeArray{Float64, 2, Date, Matrix{Float64}} 1951-12-01 to 2024-07-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1951-12-01 │  0.02513 │   104.098 │       24.19 │  1.41333 │   2.44 │ 12.527 ⋯
│ 1952-03-01 │  0.02505 │   104.824 │       23.74 │     1.43 │    2.4 │ 12.242 ⋯
│ 1952-06-01 │  0.02415 │   106.295 │       25.08 │     1.45 │   2.34 │ 12.669 ⋯
│ 1952-09-01 │  0.02685 │   104.468 │       24.26 │  1.43667 │   2.36 │ 12.131 ⋯
│ 1952-12-01 │  0.02547 │   106.402 │       26.18 │     1.41 │    2.4 │ 13.010 ⋯
│ 1953-03-01 │  0.02735 │   105.373 │       24.71 │  1.41333 │   2.43 │ 12.163 ⋯
│ 1953-06-01 │  0.02987 │    103.86 │       24.29 │     1.42 │   2.51 │ 11.750 ⋯
│ 1953-09-01 │  0.02636 │    107.81 │       23.97 │     1.43 │   2.55 │ 11.391 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [81]:
freq = 4
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)
real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi
excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
dy = log.(data_source[:Dividend]) .- log.(data_source[:Stock_Price])
term = data_source[:Yeld_10Y] .- data_source[:T_Bill] 

data = merge(excess_returns, real_t_bill, dy, term, log1p.(data_source[:T_Bill]))

start = Date(1955, 01,01)
end_d = Date(2024, 12, 31)
s_data = to(from(data, start),end_d)

display(returns_summarystats(s_data, freq))


,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_T_Bill,0.1393,0.7471,0.0586,0.2386,-1.026,-0.5885,-0.087,0.1453,0.9413,0.421,0.0773
Price_10Y_T_Bill,0.0851,0.6398,0.0108,0.1091,-0.5762,0.5559,-0.0877,-0.0183,1.491,0.1268,-0.0128
T_Bill_CPI,0.0162,0.1019,0.0059,0.0259,-0.1073,-0.3253,-0.013,0.4055,0.9784,0.3641,0.0092
Dividend_Stock_Price,0.8084,-11.0987,-14.4962,-13.2889,-17.9466,-0.1532,-15.8611,0.9766,-0.9167,-17.9326,-14.1149
Yeld_10Y_T_Bill,0.0246,0.1588,0.0546,0.0898,-0.1224,-0.2701,0.0191,0.8068,0.0302,2.2191,0.0559
T_Bill,0.0586,0.5761,0.1649,0.2223,0.0004,0.742,0.0786,0.9548,0.8548,2.8124,0.1669


nothing

In [82]:
assets_names = colnames(data)
T, n_assets = size(data)
model = VARModel(data)
model_summary(model)

,const,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill,std
Total_Stock_T_Bill,0.1355,0.1217,0.1308,1.1797,0.0277,-0.1808,-0.5434,0.067
Price_10Y_T_Bill,-0.0343,-0.1262,-0.0605,0.9065,-0.0068,0.8016,0.0603,0.0399
T_Bill_CPI,-0.0041,-0.006,0.0021,0.3007,-0.0003,0.072,0.083,0.0071
Dividend_Stock_Price,-0.1046,-0.107,-0.1413,-1.4515,0.9757,0.107,0.3704,0.0693
Yeld_10Y_T_Bill,0.0031,-0.0036,0.0151,-0.0251,0.0004,0.8235,0.014,0.0071
T_Bill,0.0015,0.0188,-0.0061,-0.0897,0.0005,0.0828,0.9791,0.0083


,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
Total_Stock_T_Bill,1.0,0.14,0.05,-0.97,0.1,-0.19
Price_10Y_T_Bill,0.14,1.0,0.17,-0.12,-0.03,-0.6
T_Bill_CPI,0.05,0.17,1.0,-0.07,-0.14,0.01
Dividend_Stock_Price,-0.97,-0.12,-0.07,1.0,-0.1,0.18
Yeld_10Y_T_Bill,0.1,-0.03,-0.14,-0.1,1.0,-0.77
T_Bill,-0.19,-0.6,0.01,0.18,-0.77,1.0


In [83]:
scenarios = simulate(model, 25*freq, 10_000)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, 0.025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  

,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
1,-0.0346,0.0082,0.0199,-17.2654,-0.0226,0.1825
5,-0.0067,0.0114,0.0058,-16.8963,0.0225,0.142
15,0.0189,0.0123,0.0021,-16.4215,0.0385,0.1198
25,0.0277,0.0112,0.0014,-16.168,0.042,0.1172


,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
1,0.1255,0.0672,0.0148,0.2744,0.0229,0.0292
5,0.059,0.0311,0.0089,0.6827,0.0264,0.0555
15,0.032,0.0142,0.0078,0.9621,0.0177,0.0665
25,0.024,0.0091,0.0077,1.0354,0.0139,0.0683


,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
1,0.0208,-0.0158,-0.0121,-0.0086,-0.059,0.0131
5,-0.0339,-0.0386,0.0345,0.0089,-0.0245,0.0454
15,0.0275,0.0234,0.0225,-0.0082,-0.0256,0.0134
25,0.0134,-0.0304,-0.0333,-0.0232,-0.0392,-0.0275


,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
1,0.0708,0.0819,0.0063,0.0842,-0.0709,0.0165
5,0.0447,0.0392,0.004,-0.0728,-0.0479,-0.008
15,0.0549,-0.0242,0.0104,0.0122,-0.0053,0.0014
25,0.0275,0.0145,0.0654,0.069,-0.0524,-0.0015


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.3294,-0.2778,-0.2405,-0.1187,-0.035,0.0499,0.1687,0.2122,0.2549
5,-0.1485,-0.1226,-0.1035,-0.0456,-0.0064,0.0325,0.0902,0.1096,0.131
15,-0.0569,-0.0435,-0.0327,-0.0027,0.0185,0.0403,0.071,0.082,0.0946
25,-0.0277,-0.0188,-0.0116,0.0115,0.0278,0.0439,0.0672,0.0753,0.084


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1511,-0.1247,-0.1033,-0.0368,0.0082,0.0531,0.1176,0.1402,0.1656
5,-0.0603,-0.0501,-0.0405,-0.0091,0.0116,0.0324,0.0616,0.0719,0.0844
15,-0.0206,-0.0153,-0.0113,0.0025,0.0122,0.0218,0.0355,0.0397,0.0449
25,-0.0105,-0.0065,-0.0038,0.0051,0.0113,0.0173,0.0262,0.0289,0.0322


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.0147,-0.0094,-0.0046,0.0099,0.0198,0.03,0.0441,0.0486,0.0546
5,-0.0149,-0.0116,-0.009,-0.0003,0.0058,0.0118,0.0206,0.0237,0.0271
15,-0.0157,-0.0131,-0.0106,-0.0032,0.0021,0.0073,0.015,0.0176,0.0202
25,-0.0165,-0.0136,-0.0113,-0.0038,0.0015,0.0067,0.0141,0.0165,0.0194


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-17.9154,-17.7999,-17.7142,-17.4509,-17.2665,-17.0832,-16.8184,-16.7236,-16.604
5,-18.4842,-18.2349,-18.0088,-17.3585,-16.9006,-16.4322,-15.7788,-15.5594,-15.2868
15,-18.6716,-18.3188,-18.0101,-17.053,-16.4345,-15.7809,-14.8321,-14.5246,-14.2017
25,-18.614,-18.2263,-17.8787,-16.856,-16.1623,-15.4757,-14.4693,-14.1508,-13.7798


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.0758,-0.0685,-0.0606,-0.0379,-0.0224,-0.0071,0.0149,0.0215,0.0283
5,-0.0393,-0.0291,-0.021,0.0049,0.0222,0.0408,0.0661,0.074,0.0831
15,-0.0033,0.0038,0.0096,0.0264,0.0387,0.0505,0.0677,0.0726,0.0793
25,0.0093,0.0149,0.0192,0.0326,0.0421,0.0516,0.0646,0.0687,0.0736


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,0.1156,0.1252,0.1346,0.1628,0.1822,0.2021,0.2308,0.2394,0.251
5,0.0156,0.0347,0.0509,0.1045,0.1413,0.1793,0.2346,0.2507,0.2721
15,-0.0332,-0.0093,0.0113,0.0742,0.1203,0.1646,0.2302,0.2506,0.2753
25,-0.0438,-0.0149,0.0066,0.07,0.1179,0.1638,0.2283,0.25,0.2766


In [84]:
real_equity_scenarios = reshape(scenarios[1, :, :] .+ scenarios[3, :,:], 1, 101, 10_000)
real_bond_scenarios = reshape(scenarios[2, :, :] .+ scenarios[3, :,:], 1, 101, 10_000)

ret_in_years = cum_returns_in_periods(vcat(real_equity_scenarios, real_bond_scenarios), periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names[1:2], string.(periods))

for a in 1:2
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, 0.025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  



,Total_Stock_T_Bill,Price_10Y_T_Bill
1,-0.0148,0.0281
5,-0.0009,0.0172
15,0.021,0.0144
25,0.0291,0.0127


,Total_Stock_T_Bill,Price_10Y_T_Bill
1,0.128,0.0719
5,0.0594,0.0306
15,0.0286,0.0127
25,0.0194,0.009


,Total_Stock_T_Bill,Price_10Y_T_Bill
1,0.0191,-0.0142
5,-0.0292,-0.0563
15,0.0272,0.0151
25,0.0011,-0.0307


,Total_Stock_T_Bill,Price_10Y_T_Bill
1,0.0574,0.0693
5,0.0499,0.0655
15,0.061,-0.0156
25,0.042,-0.0036


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.3164,-0.2629,-0.2223,-0.1006,-0.0156,0.0715,0.1943,0.238,0.2803
5,-0.1432,-0.118,-0.0987,-0.0405,-0.001,0.0386,0.0965,0.1161,0.1377
15,-0.0452,-0.0342,-0.0254,0.0014,0.021,0.0403,0.0682,0.0773,0.0883
25,-0.0161,-0.0089,-0.003,0.016,0.0292,0.042,0.061,0.0673,0.0746


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1406,-0.1146,-0.0893,-0.0201,0.0278,0.0768,0.1459,0.1697,0.1975
5,-0.055,-0.0436,-0.034,-0.0027,0.0174,0.0377,0.0665,0.0763,0.0889
15,-0.0149,-0.0103,-0.0061,0.0058,0.0143,0.0229,0.0351,0.0393,0.0443
25,-0.0082,-0.0049,-0.0021,0.0067,0.0127,0.0187,0.0273,0.0301,0.0332


In [85]:
mdd, mddl = max_drawdown_and_length(real_equity_scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(real_bond_scenarios[1,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_T_Bill

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.265715  0.363334  0.432353  0.507948  0.65022

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 18.0  30.0  41.0  55.0  88.0

:Price_10Y_T_Bill

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.184899  0.253716  0.306368  0.373933  0.501054

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 21.0  34.0  44.0  59.0  83.0